In [1]:
import os
import json
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from datetime import datetime
from cog_analysis import load_boat_data
from IPython.display import display

# --- Utility Functions ---

def load_summary_intervals(summary_file="summary.json"):
    with open(summary_file, "r") as f:
        return {r["run"]: r["intervals"] for r in json.load(f)}

def filter_interval(df, start, end):
    return df[(df["SecondsSince1970"] >= start) & (df["SecondsSince1970"] <= end)].reset_index(drop=True)

def compute_stats(df, columns):
    stats = {}
    for col in columns:
        data = df[col].dropna()
        if data.empty:
            continue
        if col == "COG":
            mean = _circular_mean(data)
        else:
            mean = data.mean()
        stats[col] = {
            "Avg": mean,
            "Min": data.min(),
            "Max": data.max(),
            "Count": len(data),
            "StdDev": data.std()
        }
    return pd.DataFrame(stats).T

def compute_distance_series(lat_series, lon_series, straight=False):
    if len(lat_series) < 2:
        return 0
    start = (lat_series.iloc[0], lon_series.iloc[0])
    end = (lat_series.iloc[-1], lon_series.iloc[-1])
    if straight:
        return geodesic(start, end).meters
    return sum(
        geodesic((lat_series.iloc[i - 1], lon_series.iloc[i - 1]),
                 (lat_series.iloc[i], lon_series.iloc[i])).meters
        for i in range(1, len(lat_series))
    )

def _circular_mean(angles):
    rad = np.radians(angles)
    return np.degrees(np.arctan2(np.mean(np.sin(rad)), np.mean(np.cos(rad)))) % 360

def _angle_to_vector(angle_deg):
    rad = np.radians(angle_deg)
    return np.array([np.sin(rad), np.cos(rad)])

def _to_meters(lat, lon, ref_lat, ref_lon):
    dx = (lon - ref_lon) * 111319.9 * np.cos(np.radians(ref_lat))
    dy = (lat - ref_lat) * 111319.9
    return np.array([dx, dy])

def _empty_result():
    return pd.DataFrame(
        [[np.nan]*3]*4,
        index=['Initial Deficit', 'Final Deficit', 'Total Gain', 'Gain per Minute'],
        columns=['Forward', 'Lateral', 'VMG']
    )

# --- Core Analysis Functions ---

def compute_directional_gain(df1, df2):
    if df1.empty or df2.empty or len(df1) < 2 or len(df2) < 2:
        return _empty_result()

    try:
        cog = _circular_mean(df1["COG"].dropna())
        twd = _circular_mean(df1["TWD"].dropna())

        forward_vec = _angle_to_vector(cog)
        perp_vec = np.array([-forward_vec[1], forward_vec[0]])
        vmg_vec = _angle_to_vector(twd)
        wind_sign = np.sign(np.dot(vmg_vec, perp_vec)) or 1

        ref_lat, ref_lon = df2["Lat"].iloc[0], df2["Lon"].iloc[0]
        pos1_start = _to_meters(df1["Lat"].iloc[0], df1["Lon"].iloc[0], ref_lat, ref_lon)
        pos1_end = _to_meters(df1["Lat"].iloc[-1], df1["Lon"].iloc[-1], ref_lat, ref_lon)
        pos2_start = _to_meters(df2["Lat"].iloc[0], df2["Lon"].iloc[0], ref_lat, ref_lon)
        pos2_end = _to_meters(df2["Lat"].iloc[-1], df2["Lon"].iloc[-1], ref_lat, ref_lon)

        initial_rel = pos1_start - pos2_start
        final_rel = pos1_end - pos2_end
        progress = initial_rel - final_rel
        duration_min = (df1["SecondsSince1970"].iloc[-1] - df1["SecondsSince1970"].iloc[0]) / 60 or np.nan

        def calc_gain(vec, sign=1):
            start_deficit = -sign * np.dot(initial_rel, vec)
            end_deficit = -sign * np.dot(final_rel, vec)
            gain = sign * np.dot(progress, vec)
            return [start_deficit, end_deficit, gain, gain / duration_min]

        return pd.DataFrame({
            "Forward": calc_gain(forward_vec),
            "Lateral": calc_gain(perp_vec, wind_sign),
            "VMG": calc_gain(vmg_vec)
        }, index=['Initial Deficit', 'Final Deficit', 'Total Gain', 'Gain per Minute'])

    except Exception as e:
        print(f"Error computing directional gain: {e}")
        return _empty_result()

def merge_stats(stats1, stats2, label1, label2):
    stats1 = stats1.rename(columns=lambda x: f"{x} ({label1})")
    stats2 = stats2.rename(columns=lambda x: f"{x} ({label2})")
    combined = pd.concat([stats1, stats2], axis=1)
    order = ["Avg", "Min", "Max", "Count", "StdDev"]
    cols = [f"{stat} ({label})" for stat in order for label in (label1, label2) if f"{stat} ({label})" in combined.columns]
    return combined[cols]

def display_all(merged_stats, summary_df, gain_table, boat1_name, boat2_name):
    # Identifier les colonnes numériques pour le formatage
    numeric_cols = merged_stats.select_dtypes(include=[np.number]).columns
    styled = style_comparative_wins(merged_stats, boat1_name, boat2_name)
    styled = styled.format({col: "{:.4g}" for col in merged_stats.select_dtypes(include=[np.number]).columns})
    styled = styled.set_caption("Run Statistics")
    display(styled)

    display(summary_df.style.format("{:.4g}").set_caption("Distance Summary"))
    display(gain_table.style.set_caption(f"{boat1_name} gains relative to {boat2_name}"))

def load_and_reduce_boat_data(run_path, summary_dict):
    csv_files = sorted(f for f in os.listdir(run_path) if f.endswith(".csv"))
    if len(csv_files) < 2:
        raise ValueError("At least two CSV files are required.")

    df1, df2, name1, name2 = load_boat_data(
        os.path.join(run_path, csv_files[0]),
        os.path.join(run_path, csv_files[1])
    )
    if df1.empty or df2.empty:
        raise ValueError("One or both boat DataFrames are empty")

    run_name = os.path.basename(run_path)
    intervals = summary_dict.get(run_name)
    if not intervals or len(intervals) < 2:
        raise ValueError(f"No or insufficient intervals for run: {run_name}")

    return {
        "full_df1": df1, "full_df2": df2,
        "reduced_boat1_int1_df": filter_interval(df1, intervals[0]["start_time"], intervals[0]["end_time"]),
        "reduced_boat2_int1_df": filter_interval(df2, intervals[0]["start_time"], intervals[0]["end_time"]),
        "reduced_boat1_int2_df": filter_interval(df1, intervals[1]["start_time"], intervals[1]["end_time"]),
        "reduced_boat2_int2_df": filter_interval(df2, intervals[1]["start_time"], intervals[1]["end_time"]),
        "boat1_name": name1, "boat2_name": name2
    }

def compare_runs(df1, df2, label1, label2):
    cols = ["TWS", "TWD", "SOG", "VMG", "COG", "TWA_Abs", "Heel_Lwd", "Side_lines", "Line_C", "Total_lines"]
    stats1 = compute_stats(df1, cols)
    stats2 = compute_stats(df2, cols)
    dist1 = compute_distance_series(df1["Lat"], df1["Lon"])
    dist2 = compute_distance_series(df2["Lat"], df2["Lon"])
    straight1 = compute_distance_series(df1["Lat"], df1["Lon"], straight=True)
    straight2 = compute_distance_series(df2["Lat"], df2["Lon"], straight=True)
    summary = pd.DataFrame({label1: [dist1, straight1], label2: [dist2, straight2]}, index=["Distance [m]", "Straight Line [m]"])
    return stats1, stats2, summary

def add_winner_columns(merged_stats, name1, name2):
    rules = {
        "SOG": {"Avg": "max", "StdDev": "min"},
        "VMG": {"Avg": "max", "StdDev": "min"},
        "COG": {"StdDev": "min"},
        "Heel_Lwd": {"Avg": "max", "StdDev": "min"},
        "Total_lines": {"Avg": "max", "StdDev": "min"},
        "Side_lines": {"Avg": "max", "StdDev": "min"},
        "Line_C": {"Avg": "max", "StdDev": "min"},
    }

    winner_avg = []
    winner_std = []
    winner_overall = []

    for index in merged_stats.index:
        rule = rules.get(index, {})
        scores = {name1: 0, name2: 0}

        avg_win, std_win = "", ""

        # Avg rule
        decisive_avg = "Avg" in rule
        if decisive_avg:
            col1 = f"Avg ({name1})"
            col2 = f"Avg ({name2})"
            if col1 in merged_stats.columns and col2 in merged_stats.columns:
                val1 = merged_stats.at[index, col1]
                val2 = merged_stats.at[index, col2]
                if pd.notna(val1) and pd.notna(val2):
                    if rule["Avg"] == "max":
                        avg_win = name1 if val1 > val2 else name2
                    else:
                        avg_win = name1 if val1 < val2 else name2
                    scores[avg_win] += 1

        # StdDev rule
        decisive_std = "StdDev" in rule
        if decisive_std:
            col1 = f"StdDev ({name1})"
            col2 = f"StdDev ({name2})"
            if col1 in merged_stats.columns and col2 in merged_stats.columns:
                val1 = merged_stats.at[index, col1]
                val2 = merged_stats.at[index, col2]
                if pd.notna(val1) and pd.notna(val2):
                    std_win = name1 if val1 < val2 else name2
                    scores[std_win] += 1

        # Final Overall Decision
        if scores[name1] > scores[name2]:
            overall = name1
        elif scores[name2] > scores[name1]:
            overall = name2
        elif scores[name1] == scores[name2] == 0:
            overall = ""
        else:
            overall = "Tie"

        # Format: decisive wins unmarked, non-decisive in parentheses
        winner_avg.append(avg_win if decisive_avg else f"({avg_win})" if avg_win else "")
        winner_std.append(std_win if decisive_std else f"({std_win})" if std_win else "")
        winner_overall.append(overall)

    merged_stats["Winner (Avg)"] = winner_avg
    merged_stats["Winner (StdDev)"] = winner_std
    merged_stats["Winner (Overall)"] = winner_overall

    return merged_stats

def style_comparative_wins(df, name1, name2):
    rules = {
        "SOG": {"Avg": "max", "StdDev": "min"},
        "VMG": {"Avg": "max", "StdDev": "min"},
        "COG": {"StdDev": "min"},
        "Heel_Lwd": {"Avg": "max", "StdDev": "min"},
        "Total_lines": {"Avg": "max", "StdDev": "min"},
        "Side_lines": {"Avg": "max", "StdDev": "min"},
        "Line_C": {"Avg": "max", "StdDev": "min"},
    }

    def highlight(row):
        styles = [""] * len(row)
        index = row.name
        rule = rules.get(index, {})
        for metric, pref in rule.items():
            col1 = f"{metric} ({name1})"
            col2 = f"{metric} ({name2})"
            if col1 in df.columns and col2 in df.columns:
                val1 = row[col1]
                val2 = row[col2]
                if pd.notna(val1) and pd.notna(val2):
                    if pref == "max":
                        if val1 > val2:
                            styles[df.columns.get_loc(col1)] = "background-color: lightgreen"
                            styles[df.columns.get_loc(col2)] = "background-color: lightcoral"
                        elif val1 < val2:
                            styles[df.columns.get_loc(col2)] = "background-color: lightgreen"
                            styles[df.columns.get_loc(col1)] = "background-color: lightcoral"
                    elif pref == "min":
                        if val1 < val2:
                            styles[df.columns.get_loc(col1)] = "background-color: lightgreen"
                            styles[df.columns.get_loc(col2)] = "background-color: lightcoral"
                        elif val1 > val2:
                            styles[df.columns.get_loc(col2)] = "background-color: lightgreen"
                            styles[df.columns.get_loc(col1)] = "background-color: lightcoral"
        return styles

    return df.style.apply(highlight, axis=1)


def process_run(df1, df2, name1, name2, title):
    if df1.empty or df2.empty:
        print(f"⚠️ Skipping {title} due to empty data: {name1} vs {name2}")
        return

    df1, df2 = df1.copy(), df2.copy()
    df1["ISODateTimeUTC"] = pd.to_datetime(df1["ISODateTimeUTC"], errors="coerce")
    df2["ISODateTimeUTC"] = pd.to_datetime(df2["ISODateTimeUTC"], errors="coerce")

    if df1["ISODateTimeUTC"].isna().all() or df2["ISODateTimeUTC"].isna().all():
        print(f"⚠️ Skipping {title} due to invalid timestamps: {name1} vs {name2}")
        return

    start_time = df1["ISODateTimeUTC"].iloc[0].ceil("min")
    df1 = df1[df1["ISODateTimeUTC"] >= start_time]
    df2 = df2[df2["ISODateTimeUTC"] >= start_time]
    if df1.empty or df2.empty:
        print(f"⚠️ Skipping {title} after time alignment")
        return

    print("\n" + "="*80)
    print(title)
    print("="*80)

    stats1, stats2, summary = compare_runs(df1, df2, name1, name2)
    merged = merge_stats(stats1, stats2, name1, name2)
    gain = compute_directional_gain(df1, df2)
    merged = add_winner_columns(merged, name1, name2)
    display_all(merged, summary, gain, name1, name2)

def process_all_run(run_path, summary_path, tot = False):
    summary_dict = load_summary_intervals(summary_path)
    data = load_and_reduce_boat_data(run_path, summary_dict)
    name1, name2 = data["boat1_name"], data["boat2_name"]
    if tot:
        print(f"Processing total run for {name1} vs {name2}")
        process_run(data["full_df1"], data["full_df2"], name1, name2, "Total Run")
    else:
        print(f"Processing only reduced intervals for {name1} vs {name2}")
        run_name = os.path.basename(run_path)
        intervals = summary_dict.get(run_name, [])
        
        start1 = datetime.utcfromtimestamp(intervals[0]["start_time"]).strftime("%Y-%m-%d %H:%M:%S")
        end1 = datetime.utcfromtimestamp(intervals[0]["end_time"]).strftime("%Y-%m-%d %H:%M:%S")
        title1 = f"Interval 1: Upwind from {start1} to {end1}: {name1} vs {name2}"
        process_run(data["reduced_boat1_int1_df"], data["reduced_boat2_int1_df"], name1, name2, title1)

        start2 = datetime.utcfromtimestamp(intervals[1]["start_time"]).strftime("%Y-%m-%d %H:%M:%S")
        end2 = datetime.utcfromtimestamp(intervals[1]["end_time"]).strftime("%Y-%m-%d %H:%M:%S")
        title2 = f"Interval 2: Downwind from {start2} to {end2}: {name1} vs {name2}"
        process_run(data["reduced_boat1_int2_df"], data["reduced_boat2_int2_df"], name1, name2, title2)

In [2]:
import os
import json
from cog_analysis import analyze_session

summary_path = "summary.json"

base_dir = "../Data_Sailnjord/Straight_lines"

# Scan de tous les dossiers de date
for date_folder in sorted(os.listdir(base_dir)):
    date_path = os.path.join(base_dir, date_folder)

    # Scan de tous les runs dans chaque dossier de date
    for run_folder in sorted(os.listdir(date_path)):
        run_path = os.path.join(date_path, run_folder)
        print("\n\nProcessing:", run_path)
        try:
            process_all_run(run_path, summary_path, tot=False)
        except ValueError as e:
            print(f"Error: {e}")



Processing: ../Data_Sailnjord/Straight_lines\06_06\06_06_Run1
Processing only reduced intervals for Gian Stragiotti vs Karl Maeder

Interval 1: Upwind from 2025-06-06 13:47:44 to 2025-06-06 13:49:43: Gian Stragiotti vs Karl Maeder


,Avg (Gian Stragiotti),Avg (Karl Maeder),Min (Gian Stragiotti),Min (Karl Maeder),Max (Gian Stragiotti),Max (Karl Maeder),Count (Gian Stragiotti),Count (Karl Maeder),StdDev (Gian Stragiotti),StdDev (Karl Maeder),Winner (Avg),Winner (StdDev),Winner (Overall)
TWS,7.165,7.165,5.204,5.204,9.395,9.394,1031,1031,1.28,1.28,,,
TWD,171.5,171.5,167.9,167.9,173.8,173.8,1031,1031,1.507,1.507,,,
SOG,20.34,20.06,16.8,17.9,22.3,22.2,1031,1031,0.9389,0.6387,Gian Stragiotti,Karl Maeder,Tie
VMG,13.71,13.43,9.711,10.02,17.37,16.49,1031,1031,1.444,1.177,Gian Stragiotti,Karl Maeder,Tie
COG,124,123.6,109.9,112.5,137.2,136.9,1031,1031,4.733,4.017,,Karl Maeder,Karl Maeder
TWA_Abs,47.44,47.9,35.49,36.52,60.84,58.91,1031,1031,4.8,3.902,,,
Heel_Lwd,55.2,53.49,36.5,34.3,73.5,76.1,1031,1031,6.227,5.986,Gian Stragiotti,Karl Maeder,Tie
Side_lines,12.94,13.5,2.2,4.9,23.7,25.5,1031,1031,3.684,3.653,Karl Maeder,Karl Maeder,Karl Maeder
Line_C,113.6,110.5,71.9,71.8,152,141,1024,1031,11.79,11.53,Gian Stragiotti,Karl Maeder,Tie
Total_lines,126.4,124.1,82.4,79.1,170.1,162.4,1031,1031,13.47,13.21,Gian Stragiotti,Karl Maeder,Tie


,Gian Stragiotti,Karl Maeder
Distance [m],1081,1066
Straight Line [m],1075,1061


,Forward,Lateral,VMG
Initial Deficit,4.392011,-14.158777,-7.458954
Final Deficit,-9.538289,-21.325081,-22.159097
Total Gain,-13.930300,-7.166304,-14.700143
Gain per Minute,-8.115447,-4.174911,-8.563938



Interval 2: Downwind from 2025-06-06 13:51:21 to 2025-06-06 13:52:50: Gian Stragiotti vs Karl Maeder


...

,Forward,Lateral,VMG
Initial Deficit,-10.168994,9.610727,0.786487
Final Deficit,-3.725766,14.412579,8.611711
Total Gain,6.443227,4.801851,7.825224
Gain per Minute,6.891398,5.135853,8.369522



Interval 2: Downwind from 2025-06-10 13:42:11 to 2025-06-10 13:43:22: Karl Maeder vs SenseBoard


,Avg (Karl Maeder),Avg (SenseBoard),Min (Karl Maeder),Min (SenseBoard),Max (Karl Maeder),Max (SenseBoard),Count (Karl Maeder),Count (SenseBoard),StdDev (Karl Maeder),StdDev (SenseBoard),Winner (Avg),Winner (StdDev),Winner (Overall)
TWS,8.101,8.101,7.705,7.705,8.7,8.7,222,222,0.264,0.2639,,,
TWD,208,208,207.4,207.4,208.6,208.6,222,222,0.3219,0.3219,,,
SOG,27.03,27.48,25.6,26.6,28.5,28.3,222,222,0.7799,0.5243,SenseBoard,SenseBoard,SenseBoard
VMG,21.21,21.68,18.95,19.67,22.78,23.52,222,222,0.9375,0.8309,SenseBoard,SenseBoard,SenseBoard
COG,349.7,350.1,345.2,343.7,355.6,356,222,222,2.084,2.402,,Karl Maeder,Karl Maeder
TWA_Abs,141.8,142.1,136.9,136,148.2,148.2,222,222,2.213,2.241,,,
Heel_Lwd,48.87,52.58,35.6,36.5,63.9,66.5,222,222,5.537,5.996,SenseBoard,Karl Maeder,Tie
Side_lines,10.53,16.04,1.3,8.8,21.1,23.3,222,222,3.035,2.562,SenseBoard,SenseBoard,SenseBoard
Line_C,85.55,104.2,55.3,75.7,117.2,125.3,221,222,12.69,10.87,SenseBoard,SenseBoard,SenseBoard
Total_lines,96.3,120.3,63,84.5,131.5,145.2,222,222,14.7,12,SenseBoard,SenseBoard,SenseBoard


,Karl Maeder,SenseBoard
Distance [m],307.5,312.8
Straight Line [m],307.3,312.5


,Forward,Lateral,VMG
Initial Deficit,23.350925,43.449911,8.553650
Final Deficit,28.635811,41.711644,3.326872
Total Gain,5.284887,-1.738267,-5.226777
Gain per Minute,14.346161,-4.718636,-14.188420
